# Load Required Libraries

In [4]:
!conda install -n python3 -y -c conda-forge keras

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.6.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs: 
    - keras


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |    h14c3975_1002         3.1 MB  conda-forge
    keras-2.2.4                |           py36_0         456 KB  conda-forge
    astor-0.7.1                |             py_0          22 KB  conda-forge
    gast-0.2.1.post0           |             py_0           9 KB  conda-forge
    pygpu-0.7.6                |py36h3010b51_1000         657 KB  conda-forge
    c-ares-1.15.0              |    h14c3975_1001          98 KB  conda-forge
    theano-1.0.3               |           py36_0       

In [5]:
!conda install -n python3 -y -c conda-forge xgboost

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.6.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs: 
    - xgboost


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    xgboost-0.81               |py36hf484d3e_1000           9 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    libstdcxx-ng-7.3.0         |       hdf63c60_0         2.6 MB  conda-forge
    py-xgboost-0.81            |py36hf484d3e_1000          66 KB  conda-forge
    libxgboost-0.81            |    hf484d3e_1000         3.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         6.4 MB

T

In [7]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score 
from sklearn.metrics import roc_auc_score,log_loss
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split,KFold,cross_val_predict,train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.decomposition import TruncatedSVD
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.svm import SVC
#import gensim
from xgboost import XGBClassifier
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.optimizers import Adam
import nltk

Using TensorFlow backend.


In [8]:
import boto3
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()
bucket='stocknewsstorage'
data_key = 'AllDataProcessed2.xlsx'
data_location = 's3://{}/{}'.format(bucket, data_key)
data = pd.read_excel(data_location)
print(data.head(10))
data.describe(include='all')

        Date company                                               news  \
0 2019-01-14     HCL  HCL Tech Q3 PAT seen up 4.2% QoQ to Rs. 2,629....   
1 2018-12-07     HCL  HCL Technologies-IBM deal fails to enthuse inv...   
2 2018-12-06     HCL  Technical Views | Top buy & sell ideas by Ashw...   
3 2018-11-09     HCL  HCL Technologies Limited Q2 FY’19 Earnings Con...   
4 2018-10-29     HCL  Buy HCL Technologies, target Rs 1182: Anand Rathi   
5 2018-10-24     HCL  HCL Technologies – good outlook backed by reas...   
6 2018-10-23     HCL  HCL Technologies sees 5.7% sequential rise in ...   
7 2018-10-22     HCL  HCL Technologies Q2 results on October 23; her...   
8 2018-10-15     HCL  Stock Picks of the Day | Nifty may be vulnerab...   
9 2018-10-08     HCL  HCL Tech to set up global IT centres in Andhra...   

   loss/profit   %change                                     Processed_news  
0            0 -0.472399  hcl tech q3 pat see up 4 . 2 % qoq rs . 2,629 ...  
1            0 -3.

,Date,company,news,loss/profit,%change,Processed_news
count,9235,9235,9235,9235.000000,9235.000000,9235
unique,3078,8,8828,NaN,NaN,8821
top,2007-09-27 00:00:00,WIPRO,6 trading stocks for short term gain,NaN,NaN,6 trading stock short term gain
freq,8,1371,7,NaN,NaN,7
first,2005-01-03 00:00:00,NaN,NaN,NaN,NaN,NaN
last,2019-01-18 00:00:00,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,0.486952,-0.009986,NaN
std,NaN,NaN,NaN,0.499857,2.255059,NaN
min,NaN,NaN,NaN,0.000000,-20.791906,NaN
25%,NaN,NaN,NaN,0.000000,-1.104271,NaN


In [9]:
X = data["Processed_news"]
y = data["loss/profit"]

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2019)

# Function to call various Algorithm

In [11]:
def evaluate_features(X, y, clf=None):
    """General helper function for evaluating effectiveness of passed features in ML model
    
    Prints out Log loss, accuracy, and confusion matrix with 3-fold stratified cross-validation
    
    Args:
        X (array-like): Features array. Shape (n_samples, n_features)
        
        y (array-like): Labels array. Shape (n_samples,)
        
        clf: Classifier to use. If None, default Log reg is use.
    """
    if clf is None:
        clf = LogisticRegression()
    
    probas = cross_val_predict(clf, X, y, cv=KFold(random_state=2019,n_splits=6), 
                              n_jobs=-1, method='predict_proba')
    pred_indices = np.argmax(probas, axis=1)
    classes = np.unique(y)
    preds = classes[pred_indices]
    print('Log loss: {}'.format(log_loss(y, probas)))
    print('Accuracy: {}'.format(accuracy_score(y, preds)))
    print('Classification report',classification_report(y,preds))
    print('Confusion Matrix  ', confusion_matrix(y,preds))

# Using Count Vectorizer

In [12]:
count_vectorizer = CountVectorizer(ngram_range=(1,2))  

bag_of_words = count_vectorizer.fit_transform(data['Processed_news'])
    
len(count_vectorizer.get_feature_names())

58527

In [13]:
evaluate_features(bag_of_words, data['loss/profit'])

Log loss: 0.8019118033858804
Accuracy: 0.5449918787222523
Classification report              precision    recall  f1-score   support

          0       0.55      0.58      0.57      4738
          1       0.53      0.51      0.52      4497

avg / total       0.54      0.54      0.54      9235

Confusion Matrix   [[2732 2006]
 [2196 2301]]


In [14]:
evaluate_features(bag_of_words, data['loss/profit'], 
                  RandomForestClassifier(n_estimators=300, max_depth=30,random_state=2019))

Log loss: 0.6895947799462351
Accuracy: 0.5378451543042772
Classification report              precision    recall  f1-score   support

          0       0.53      0.82      0.65      4738
          1       0.56      0.24      0.33      4497

avg / total       0.55      0.54      0.49      9235

Confusion Matrix   [[3905  833]
 [3435 1062]]


In [15]:
evaluate_features(bag_of_words, data['loss/profit'],SVC(kernel='linear',probability=True,random_state=2019))

Log loss: 0.6887768641156938
Accuracy: 0.5458581483486735
Classification report              precision    recall  f1-score   support

          0       0.54      0.76      0.63      4738
          1       0.56      0.32      0.41      4497

avg / total       0.55      0.55      0.52      9235

Confusion Matrix   [[3591 1147]
 [3047 1450]]


# Using TF-IDF

In [16]:
count_vectorizer = TfidfVectorizer()    

tfidf = count_vectorizer.fit_transform(data['Processed_news'])

len(count_vectorizer.get_feature_names())

8391

In [17]:
evaluate_features(tfidf, data['loss/profit'])

Log loss: 0.6956253952558793
Accuracy: 0.5408770979967515
Classification report              precision    recall  f1-score   support

          0       0.55      0.60      0.57      4738
          1       0.53      0.48      0.50      4497

avg / total       0.54      0.54      0.54      9235

Confusion Matrix   [[2842 1896]
 [2344 2153]]


In [18]:
evaluate_features(tfidf, data['loss/profit'], SVC(kernel='linear', probability=True, random_state=2019))

Log loss: 0.6886725767043995
Accuracy: 0.542609637249594
Classification report              precision    recall  f1-score   support

          0       0.54      0.73      0.62      4738
          1       0.55      0.35      0.42      4497

avg / total       0.54      0.54      0.53      9235

Confusion Matrix   [[3454 1284]
 [2940 1557]]


In [13]:
evaluate_features(tfidf, data['loss/profit'], SVC(kernel='rbf', probability=True, random_state=45,C=2,gamma=.01))

Log loss: 0.6897448911893616
Accuracy: 0.5431510557661072
Classification report               precision    recall  f1-score   support

           0       0.54      0.78      0.64      4738
           1       0.56      0.30      0.39      4497

   micro avg       0.54      0.54      0.54      9235
   macro avg       0.55      0.54      0.51      9235
weighted avg       0.55      0.54      0.51      9235

Confusion Matrix   [[3689 1049]
 [3170 1327]]


# LightGBM

In [19]:
basictf= TfidfVectorizer(use_idf=True, min_df=0.0, max_df=1.0, ngram_range=(1,2),sublinear_tf=True)
basicTrain=basictf.fit_transform(X_train)
print(basicTrain.shape)

(7388, 49998)


In [102]:
d_train = lgb.Dataset(basicTrain, label=y_train)
params = {}
params['learning_rate'] = 0.002
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 32
params['min_data'] = 50
params['max_depth'] = 20
params['max_bin']=1024
clf = lgb.train(params, d_train,1000)

In [103]:
basicTest = basictf.transform(X_test)
y_pred = clf.predict(basicTest)
y_pred

array([0.42222875, 0.48595289, 0.44494029, ..., 0.51849818, 0.41840873,
       0.48488979])

In [106]:
for i in range(0,2227):
    if y_pred[i]>=.5:       # setting threshold to .5
       y_pred[i]=1
    else:  
       y_pred[i]=0

In [107]:
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_pred,y_test))

[[770 366]
 [653 438]]
0.5424337674000899


In [6]:
class MySentences(object):
    """MySentences is a generator to produce a list of tokenized sentences 
    
    Takes a list of numpy arrays containing documents.
    
    Args:
        arrays: List of arrays, where each element in the array contains a document.
    """
    def __init__(self, *arrays):
        self.arrays = arrays
 
    def __iter__(self):
        for array in self.arrays:
            for document in array:
                for sent in nltk.sent_tokenize(document):
                    yield nltk.word_tokenize(sent)

def get_word2vec(sentences, location):
    """Returns trained word2vec
    
    Args:
        sentences: iterator for sentences
        
        location (str): Path to save/load word2vec
    """
    if os.path.exists(location):
        print('Found {}'.format(location))
        model = gensim.models.Word2Vec.load(location)
        return model
    
    print('{} not found. training model'.format(location))
    model = gensim.models.Word2Vec(sentences, size=1000, window=5, min_count=5, workers=4)
    print('Model done training. Saving to disk')
    model.save(location)
    return model    

In [7]:
w2vec = get_word2vec(
    MySentences(
        data['Processed_news'].values, 
    ),
    'w2vmodel'
)

Found w2vmodel


In [10]:
class MyTokenizer:
    def __init__(self):w2vec = get_word2vec(
    MySentences(
        data['Processed_news'].values, 
    ),
    'w2vmodel'
)
    pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)
    
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.syn0[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)    

In [13]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2vec)
mean_embedded = mean_embedding_vectorizer.fit_transform(data['Processed_news'])
mean_embedded

C:\Users\shanthi\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).


Found w2vmodel


array([[-0.05764131,  0.03496192,  0.02898565, ...,  0.02233779,
        -0.03433947,  0.07995956],
       [-0.05459464,  0.03316778,  0.02726857, ...,  0.02107938,
        -0.03246247,  0.07540307],
       [-0.06403969,  0.03862501,  0.03191711, ...,  0.02499009,
        -0.03799589,  0.08837906],
       ...,
       [-0.04040901,  0.02406988,  0.0200002 , ...,  0.01590047,
        -0.02378608,  0.05545349],
       [-0.0430277 ,  0.02577958,  0.02139688, ...,  0.01666953,
        -0.02555102,  0.05874644],
       [-0.0565276 ,  0.03399789,  0.02814336, ...,  0.02198508,
        -0.03353221,  0.07792477]])

In [14]:
evaluate_features(mean_embedded, data['loss/profit'])

Log loss: 0.6928161544595319
Accuracy: 0.5127233351380617
Classification report               precision    recall  f1-score   support

           0       0.51      0.98      0.67      4738
           1       0.49      0.02      0.04      4497

   micro avg       0.51      0.51      0.51      9235
   macro avg       0.50      0.50      0.36      9235
weighted avg       0.50      0.51      0.37      9235

Confusion Matrix   [[4641   97]
 [4403   94]]


In [15]:
evaluate_features(mean_embedded, 
                  data['loss/profit'],
                  XGBClassifier(max_depth=4,
                                
                                )
                 )

Log loss: 0.6999228327191709
Accuracy: 0.5206280454791554
Classification report               precision    recall  f1-score   support

           0       0.53      0.60      0.56      4738
           1       0.51      0.44      0.47      4497

   micro avg       0.52      0.52      0.52      9235
   macro avg       0.52      0.52      0.52      9235
weighted avg       0.52      0.52      0.52      9235

Confusion Matrix   [[2836 1902]
 [2525 1972]]


In [16]:
evaluate_features(mean_embedded, data['loss/profit'],RandomForestClassifier(n_estimators=200, max_depth=30, random_state=45))

Log loss: 0.7293040620212402
Accuracy: 0.5204114780725501
Classification report               precision    recall  f1-score   support

           0       0.53      0.58      0.56      4738
           1       0.51      0.45      0.48      4497

   micro avg       0.52      0.52      0.52      9235
   macro avg       0.52      0.52      0.52      9235
weighted avg       0.52      0.52      0.52      9235

Confusion Matrix   [[2762 1976]
 [2453 2044]]


In [25]:
# Use the Keras tokenizer
num_words = 2000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data['Processed_news'])

In [26]:
# Pad the data 
X = tokenizer.texts_to_sequences(data['Processed_news'].values)
X = pad_sequences(X, maxlen=200)
X.shape[1]

200

In [27]:
# Build out our simple LSTM
embed_dim = 138
lstm_out = 200

# Model saving callback
ckpt_callback = ModelCheckpoint('keras_model', 
                                 monitor='val_loss', 
                                 verbose=1, 
                                 save_best_only=True, 
                                 mode='auto')

model = Sequential()
model.add(Embedding(num_words, embed_dim, input_length = X.shape[1]))
model.add(LSTM( lstm_out,recurrent_dropout=0.2, dropout=0.2,activation='relu'))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 200, 138)          276000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 200)               271200    
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 402       
Total params: 547,602
Trainable params: 547,602
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
Y = pd.get_dummies(data['loss/profit']).values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42, stratify=Y)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(7388, 200) (7388, 2)
(1847, 200) (1847, 2)


In [34]:
batch_size = 32
model.fit(X_train, Y_train, epochs=5, batch_size=batch_size, validation_split=0.1, verbose=1,shuffle=True)

Train on 6649 samples, validate on 739 samples
Epoch 1/5
6649/6649 [==============================] - 119s 18ms/step - loss: 0.6629 - acc: 0.6630 - val_loss: 0.7977 - val_acc: 0.5034
Epoch 2/5
6649/6649 [==============================] - 114s 17ms/step - loss: 0.6079 - acc: 0.6745 - val_loss: 0.8322 - val_acc: 0.5189
Epoch 3/5
6649/6649 [==============================] - 115s 17ms/step - loss: 0.5630 - acc: 0.6860 - val_loss: 0.9357 - val_acc: 0.5108
Epoch 4/5
6649/6649 [==============================] - 115s 17ms/step - loss: 0.5302 - acc: 0.7033 - val_loss: 1.0672 - val_acc: 0.5027
Epoch 5/5
6649/6649 [==============================] - 115s 17ms/step - loss: 0.4976 - acc: 0.7282 - val_loss: 1.1746 - val_acc: 0.5149


In [35]:
model.save('keras_model')
model = load_model('keras_model')

In [36]:
probas = model.predict(X_test)

In [37]:
pred_indices = np.argmax(probas, axis=1)
print(pred_indices)
classes = np.array(range(1,3))
preds = classes[pred_indices]
print(preds)
print('Log loss: {}'.format(log_loss(classes[np.argmax(Y_test, axis=1)], probas)))
print('Accuracy: {}'.format(accuracy_score(classes[np.argmax(Y_test, axis=1)], preds)))
confusion_matrix(classes[np.argmax(Y_test, axis=1)], preds)

[0 1 1 ... 1 1 1]
[1 2 2 ... 2 2 2]
Log loss: 1.0431785102003754
Accuracy: 0.5230102869518137


array([[469, 479],
       [402, 497]], dtype=int64)